In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

from IPython.display import Image, clear_output
import pandas as pd

from pprint import pprint

## Run scenarios


In [2]:
def run_scenario_n_times_correct(
    n_scenarios=10,
    expert_driving=True,
    use_same_seed=False,
    gif: str = "",
    user_config: dict = {},
) -> list:
    """
    Runs same scenario n time and collects the traces
    """

    traces = []
    config = user_config | {
        "map": 2,
        "num_scenarios": n_scenarios,
    }

    try:

        for rep in range(n_scenarios):

            env = MetaDriveEnv(config=config)
            seed = 0 if use_same_seed else rep

            obs, step_info = env.reset(seed)
            step_info["repetition"] = rep
            traces.append(step_info)
            print(f"{env.current_seed = }")
            while True:

                # get action from expert driving, or a dummy action
                action = (
                    expert(env.agent, deterministic=True)
                    if expert_driving
                    else [0.1, 0.33]
                )
                obs, reward, tm, tr, step_info = env.step(action)
                step_info["repetition"] = rep
                traces.append(step_info)

                if gif:
                    env.render(
                        mode="topdown",
                        # get the overview of the scene
                        film_size=(1000, 1000),
                        screen_size=(1000, 1000),
                        # set camer to map center
                        camera_position=env.current_map.get_center_point(),
                        # auto determine the number of pixels for 1 meter
                        scaling=None,
                        # do not pop window
                        screen_record=True,
                        window=False,
                    )

                if tm or tr:
                    break

            print(f"{gif = }")
            if gif:
                env.top_down_renderer.generate_gif(
                    gif_name=f"{gif}_seed_{rep}.gif", duration=30
                )

            env.close()
    finally:
        pass

    return traces

In [3]:
def are_traces_deterministic(traces) -> bool:
    df = pd.DataFrame(traces)

    # grouping by repetition to get a list of traces
    traces = df.groupby("repetition")

    # drop index and repetition ID to compare only step info later
    stripped_traces = [
        trace.reset_index(drop=True).drop("repetition", axis=1) for _, trace in traces
    ]

    # iterate over each trace and check if it is equal to the first one
    are_equal_to_first_trace = [
        trace.equals(stripped_traces[0]) for trace in stripped_traces
    ]

    # if all traces are equal to the first, then all traces are equal hence deterministic
    return all(are_equal_to_first_trace)

## Determinism check


In [4]:
same_seed = True
traces = run_scenario_n_times_correct(
    n_scenarios=10,
    use_same_seed=same_seed,
)
clear_output()
if same_seed:
    assert are_traces_deterministic(traces), "Traces must be deterministic!"

df = pd.DataFrame(traces)
# df

## Fidelity parameters

### Default settings

- `dt = 0.02`
- `decision_repeat = 5`
- Step time is `dt * decision_repeat = 0.1` which is 10FPS


In [5]:
# low_fidelity = run_scenario_n_times_correct(
#     n_scenarios=5, user_config={"decision_repeat": 10}, gif=True, use_same_seed=False
# )
clear_output()

In [6]:
# assert are_traces_deterministic(low_fidelity), "Traces must be deterministic!"
# df = pd.DataFrame(low_fidelity)

In [7]:
for dr in [5, 10, 15, 20]:
    traces = run_scenario_n_times_correct(
        n_scenarios=5,
        user_config={"decision_repeat": dr},
        gif=str(dr),
        use_same_seed=False,
    )

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 5


env.current_seed = 0


[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.


gif = '5'


[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 5


...

gif = '20'


[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 5


env.current_seed = 4


[INFO] Episode ended! Scenario Index: 4 Reason: arrive_dest.


gif = '20'
